<a href="https://colab.research.google.com/github/massinoLight/tp_AlgoBD/blob/main/TP_Kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
pip install pyspark

##On charge le fichier de data set

In [2]:
import requests 
file_url = "https://raw.githubusercontent.com/apache/spark/master/data/mllib/kmeans_data.txt"
	
r = requests.get(file_url, stream = True) 

with open("/content/sample_data/kmeans_data.txt", "wb") as file: 
	for block in r.iter_content(chunk_size = 1024): 
		if block: 
			file.write(block) 

##Spark context

In [10]:
from numpy import array
from math import sqrt
import sys

from pyspark import SparkContext
from pyspark.mllib.clustering import KMeans, KMeansModel


sc = SparkContext(appName="KMeansExample")  

##Determiner notre modéle (nb cluster=3,nb iteration=10)

In [26]:
data = sc.textFile("/content/sample_data/kmeans_data.txt")
parsedData = data.map(lambda line: array([float(x) for x in line.split(' ')]))

clusters = KMeans.train(parsedData, 3, maxIterations=10, initializationMode="random")

def error(point):
        center = clusters.centers[clusters.predict(point)]
        return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

    

    



Within Set Sum of Squared Error = 0.519615242270665


In [ ]:
clusters.save(sc, "model/Kmeans/KMeansModel")
sameModel = KMeansModel.load(sc, "model/Kmeans/KMeansModel")

In [11]:
sc.stop()

##Exemple 2

##Load data

In [30]:
import requests 
file_url = "https://raw.githubusercontent.com/massinoLight/tp_AlgoBD/main/3D_spatial_network.txt"
	
r = requests.get(file_url, stream = True) 

with open("/content/sample_data/3D_spatial_network.txt", "wb") as file: 
	for block in r.iter_content(chunk_size = 1024): 
		if block: 
			file.write(block) 

##Fonction pour préparer le jeu de données

In [31]:
def parseVector(line):
    return np.array([float(x) for x in line.split(',')])



In [34]:
sc = SparkContext(appName="KMeans")


##test du modéle sur le 2nd jeu de données (nb cluster=3,Max iteration=20)

In [37]:
lines = sc.textFile("/content/sample_data/3D_spatial_network.txt")
data = lines.map(parseVector)
k = 3
model = KMeans.train(data, k, maxIterations=20, initializationMode="random")
print("Final centers: " + str(model.clusterCenters))
print("Total Cost: " + str(model.computeCost(data)))

Final centers: [array([9.90633422e+07, 9.68557343e+00, 5.70686553e+01, 2.11937929e+01]), array([3.43385461e+07, 9.81595312e+00, 5.71211049e+01, 1.97624911e+01]), array([1.38119067e+08, 9.75145608e+00, 5.70835976e+01, 2.54127126e+01])]
Total Cost: 5.20328439270565e+19


In [33]:
sc.stop()